# *League of Legends* Competitive Match Data

⚠️ **Note** ⚠️ You'll probably want to be at least a little bit familiar with [*League of Legends*](https://en.wikipedia.org/wiki/Video_game) and its terminology to use this data set. If not, one of the other data sets may be more interesting to you.

Welcome to Summoner's Rift!

In this project, you will look at a large data set of information on players and teams from over 10,000 *League of Legends* competitive matches. There are many questions that can be answered using this data, such as:
- Looking at [tier one professional leagues](https://en.wikipedia.org/wiki/List_of_League_of_Legends_leagues_and_tournaments), which league has the most "action-packed" games? Is the amount of "action" in this league significantly different than in other leagues? Note that you'll have to come up with a way of quantifying "action".
- Which competitive region has the highest win rate against teams outside their region? Note you will have to find and merge region data for this question as the dataset does not have it.
- Which role "carries" (does the best) in their team more often: ADCs (Bot lanes) or Mid laners?
- Is (DSC 80 tutor) Costin's favorite champion (Talon) more likely to win or lose a match?

Of course, you are encouraged to pursue a question that is not listed above.



### Getting the Data
The dataset can be found on the [Oracle's Elixir](https://oracleselixir.com/tools/downloads) website.

You can use the most up-to-date match data file with more than 100,000 rows. You may use the older datasets if you wish, but keep in mind that League of Legends changes significantly between years. This can make it difficult to combine data from different years, or to make comparisons between years.

### Cleaning and EDA
- Clean the data.
    - Note that each `gameid` corresponds to up to 12 rows -- one for each of the 5 players on both teams and 2 containing summary data for the two teams (try to find out what distinguishes those rows). After selecting the line of inquiry, make sure to remove either the player rows or the team rows so as not to have issues later in your analysis.
    - Many columns should be of type `bool` but are not.
- Understand the data in ways relevant to your question using univariate and bivariate analysis of the data as well as aggregations.

### Assessment of Missingness
- Assess the missingness per the requirements in `project03.ipynb`

### Hypothesis Test / Permutation Test
Find a hypothesis test or permutation test to perform. You can use the questions at the top of the notebook for inspiration.

# Summary of Findings

### Introduction
On November 20th, 2019, the **Dragon soul system** was introduced to League of Legends. Simply speaking, Dragons are a neutral monster in the game that the two teams could compete for. The team which kills them could get buffs (beneficial boost) which can help them win the game. When a team kills their fourth dragon, they would get the Dragon soul, which is an even more powerful buff that **can offer a huge advantage to the team**. There are **six different types of dragons** (and one of them, chemtechs, was removed in the beginning of this year, so there are only five in the most part of this year), and each of them offers different buffs. **Dragon soul can only be gained once per game, meaning only one team can get it.**

One thing that League players sometimes discuss about dragons is that the Cloud Dragon soul's buff seems less useful than other Dragon souls based on the attributes they offer. Besides, according to the professional esport matches data, the average win rate of a team that gets Cloud Dragon souls is also lower than the average win rate of teams with other Dragon souls. To resolve this problem, the game patch 12.14 was released to adjust and balance the buffs offered by the various Dragons. **So is gaining Cloud Dragon soul really less helpful for the team to get victory than other Dragon souls? Or is it by random chance? If so, did the balance patch 12.14 fix this problem?** We will try to figure this out by the following data analysis steps.

### Cleaning and EDA
The dataset downloaded from [Oracle's Elixir](https://oracleselixir.com/tools/downloads) website is kind of messy. There are 123 columns at total, so I removed most of the columns, and only kept the colums of some basic info, like the result, and those relevant to dragons, such as the numbers of each type of dragons in the match.

Next, most of the rows in the dataset is about each player's personal data, like thier killing and deaths. We only care about the dragon statistics, which is the team data. So we removed all the rows containing players' data. Another cleaning I did is convert the values of results, consisting of `1 (win) and 0 (lose)`, to `True and False`. Some of the matches have the number of each type of dragons missing, instead, they are stored together as the total dragon numbers, in a single column where their types are indistinguishable. In this EDA section, we just want to make some intuition about our data, so I temporarily ignored these missingness, and leave them to the next section. The final cleaning I did is to creat a new column with the type of Dragons souls stored as String, instead of having several columns with the number of each type of dragons.

In my EDA process, I have the following findings: the average win rate of having the Cloud dragon soul is `67.44%`, whereas the overall win rate is `79.35%`. Also I found out that after rebalance patch 12.14, the Cloud soul win rate was increased to `100%`.

### Assessment of Missingness
Previously, we found out that some rows have missing value on the columns of number of dragons of each type, and instead the total numbers of dragons were stored in a column `dragon (type unknown)`. Personally, I think they are not NMAR, because it's not reasonable that the match organizer decided to not record the dragon types depending on the dragon types themselves. More possible explanations would be that the dragon types are not so important compared to other information, so they just did not record it.

By doing permutation test, I got the p value of `0.058`. So I am pretty confident that the the missing in dragon types columns are not independent on the win rate. This implies the win rate and the missing has some kind of relationships.

### Hypothesis Test
By doing hypothesis test with the **Null hypothesis**: Cloud Dragon soul is as helpful as other Dragon souls, the differences on the win rate is due to random chance. The actual win rate after gaining a Cloud Dragon soul is the **same** as the overall mean win rate of gaining Dragon soul. And test statistics to be the mean, with p value of `0.04478`, I **failed to reject** the null hypothesis.

# Code

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'retina'  # Higher resolution figures

### Cleaning and EDA

Firstly, let's download the dataset from the [Oracle's Elixir](https://oracleselixir.com/tools/downloads) website. I picked the 2022 Match Data, which is a CSV file containing all the League of Legends esport matches data in 2022. Then let's load it into a pandas Dataframe, and check the first few rows:

In [ ]:
full_matches = pd.read_csv('2022_LoL_esports_match_data_from_OraclesElixir_20221108.csv')
full_matches.head()

We can see that there are so many columns (123 at total) in our dataset, and most of them are irrelevant to our question of Dragon souls.

In [ ]:
print('Total number of columns: ',len(full_matches.columns))
print(list(full_matches.columns))

So let's only keep the relevant columns about dragons, victory, and the some basic information. Then we check our dataset again:

In [ ]:
dragons = full_matches[['gameid','result','patch','side','position','elementaldrakes','opp_elementaldrakes','infernals','mountains','clouds','oceans','chemtechs','hextechs','dragons (type unknown)']]
dragons.head()

It's interesting that all the dragon information seems to be `NaN`. This is because the number of dragons taken is considered to be team data rather than player data, and they are stored in rows where `position` is `team`. So we'll only keep the rows that `position` is `team`. 

In [ ]:
team_data = dragons[full_matches['position'] == 'team']
team_data.head()

Notice that the row with index `34` has `NaN` in almost every dragon columns, and only `dragons (type unknown)` column has a meaningful value. We can make a guess that maybe the organizers of some matches did not record the type of the dragons, and only the total number of dragons was recorded. We will leave these missingnesses to be handled in the **Assessment of Missingness** section. For now let's temporarily remove these missingnesses, and do some exploratory data analysis on the dataset that we've already know the Dragon soul types. 

Also, the `result` column is recorded as `1` and `0`. We will convert them into `True`s and `False`s.

In [ ]:
team_data = team_data[team_data['elementaldrakes'].notna()]
team_data['result'] = team_data['result'].astype(bool)
team_data

For the first question we have, we want to know the win rate of the teams which gained different Dragon souls. Since Dragon souls might not even appear in a match when one team wins before any side get their fourth dragon, we want another DataFrame that only contains matches when the Dragon soul appears(i.e. whether number of any dragon is `4`) and their types.

In [ ]:
dragon_souls = pd.DataFrame()

# loop through the six different types of dragons, check if their number equals to 4
for c in ['infernals','mountains','clouds','oceans','chemtechs','hextechs']:
    df = team_data[team_data[c] == 4]
    df = df.iloc[:,:5] # we only need the first five columns which contains the basic info
    df['dragon soul type'] = c # store the dragon soul type
    dragon_souls = pd.concat([dragon_souls, df]) # concat the dataframes together

dragon_souls.head()

Now we can easily group the matches by the type of dragon souls, and make some observations.

First, we want to do some **univariate analysis**, which is to look at the `win rate` and `dragon soul types` separately.

In [ ]:
avg_dragon_soul_win_rate = (dragon_souls['result'].mean()*100).round(2)
print('The average win rate when a team has a Dragon soul:', avg_dragon_soul_win_rate, '%')
[
    dragon_souls['dragon soul type']
    .value_counts()
    .plot(kind='barh',title='Frequency of Occurrence of Different Dragon Souls')
]

We see that the occurency of Chemtechs souls is significantly less than other five types of dragon souls. As mentioned in the introduction section, this is because Riot Game removed the Chemtech Dragon from the game at the beginning of this year, so they are not obtainable in the most part of this year. We also see that the average win rate of a team when it has gained a Dragon soul is very high. This is not surprising because: 1) the team which is already in advantage in early game is more likely to win the competition of killing dragons, and 2) gaining the dragon souls makes the team furthur in advantage for the powerful buff it offers.

Then let's find something more interesting by **bivariate analysis**, by combining the `win rate` and `type of dragon souls` together.

In [ ]:
dragon_souls.groupby('dragon soul type')['result'].mean().plot(kind='barh',title='Win Rate by Dragon Souls')

We see that the `win rate` of Cloud souls(around `67%`) and Chemtech souls(`50%`) which is much lower than other four(around `80%`). 

To rebalance the power of the five dragons(Chemtech was removed), Riot Game released the game patch `12.14` to adjust the buffs they offer. Let's see whether the `patch 12.14` fixed this issue by **aggregating** the data we have:

In [ ]:
win_rate_by_patch = pd.pivot_table(data=dragon_souls,values='result',index='patch',columns='dragon soul type',aggfunc='mean')
win_rate_by_patch.head()

In [ ]:
before_partch = win_rate_by_patch.iloc[:14].mean()
after_patch = win_rate_by_patch.iloc[14:].mean()
pd.concat([before_partch,after_patch],axis=1)

Despite the dataset of matches after the patch `12.14` is small, we can still see that the Cloud soul has a very high win rate afterwards.

Are all these observed statics really the case? Or they appear because of random chances? Let's continue or analysis by handling the missing values first.

### Assessment of Missingness

We've seen that some rows have number of dragons stored in the column `dragon (type unknown)` instead of recorded according to their types separately. This make it difficult to know whether Dragon souls were gained in those matches. What we can do instead is to check if these values are missing at random or not. So we'll pick all the matches that the total number of dragons (any type) is greater than `4` (smallest number that Dragon souls appears), and do a permutation test to see the relationships between the win rate and value being missing.

Firstly, let's go back to the original `full_matches`, and keep only the matches where more than 4 dragons appeared. Then compare the absolute difference of mean between unmissing and missing values to see if they come from the same data generating process.

In [ ]:
# keeping only the rows where either 'elementaldrakes' equals or greater than 4 
# or 'dragons (type unknown)' equals or greater than 4
dragons = full_matches[['gameid','result','patch','side','position','elementaldrakes','dragons (type unknown)']]
team_data = dragons[full_matches['position'] == 'team']
at_least_4_dragon = team_data[(team_data['elementaldrakes'] >= 4) | (team_data['dragons (type unknown)'] >= 4)]
at_least_4_dragon.head()

In [ ]:
# then we get the observed statistics, which is the absolute difference of mean
notna_mean = at_least_4_dragon[at_least_4_dragon['elementaldrakes'].notna()]['result'].mean()
na_mean = at_least_4_dragon[at_least_4_dragon['elementaldrakes'].isna()]['result'].mean()
observed_stat = abs(notna_mean - na_mean)
observed_stat

Then we'll do a permutation test to check the missingness mechanism of the data.

In [ ]:
repetitions = 500

perm_stats = []
for i in range(repetitions):

    # Shuffle the 'elementaldrakes' column
    shuffled_dragons = (
        at_least_4_dragon['elementaldrakes']
        .sample(frac=1)
        .reset_index(drop=True)
    )
    
    # Put them in a DataFrame
    shuffled = (
        at_least_4_dragon
        .assign(**{'shuffled elementaldrakes': shuffled_dragons})
    )
    
    # Compute the test statistic
    notna_mean = shuffled[shuffled['shuffled elementaldrakes'].notna()]['result'].mean()
    na_mean = shuffled[shuffled['shuffled elementaldrakes'].isna()]['result'].mean()
    stat = abs(notna_mean - na_mean)
    
    # Store the result
    perm_stats.append(stat)
    
perm_stats[:10]

In [ ]:
title = 'Absolute Mean Differences in Win Rate (Missing vs. Non-Missing Dragon Types)'
pd.Series(perm_stats).plot(kind='hist', density=True, ec='w', bins=10, title=title)
plt.axvline(x=observed_stat, color='red', linewidth=3);
print('P-value:', np.count_nonzero(observed_stat<=np.array(perm_stats))/repetitions)

The p value is very close to `0.05`, so we are **not** so **confident** that the missing of dragon types is independent on the match results.

### Hypothesis Test

Anyway, let's try to answer our question: **Is gaining Cloud Dragon soul really less helpful for the team to get victory than other Dragon souls?** Firstly we need to write down the null and alternative hypothesis.

**Null hypothesis**: Cloud Dragon soul is as helpful as other Dragon souls, the differences on the win rate is due to random chance. The actual win rate after gaining a Cloud Dragon soul is the **same** as the overall mean win rate of gaining Dragon soul.

**Alternative hypothesis**: Cloud Dragon soul is not so helpful compared to other Dragon souls. The actual win rate after gaining a Cloud Dragon soul is **less** than the overall mean win rate of gaining Dragon soul.

Since our null hypothesis assume that all the Dragon souls share a common win rate, so our hypothesis test is just comparing the mean win rate of Cloud souls and the overall win rate. Therefore we choose our **test statistic** to be the **mean win rate**. And we will use the standard **significance level** of `0.05`.

resulting p-value and results

In [ ]:
# Getting observed statistics, test size, and overall average win rate
observed_mean = dragon_souls[dragon_souls['dragon soul type'] == 'clouds']['result'].mean()
size = dragon_souls[dragon_souls['dragon soul type'] == 'clouds'].shape[0]
win_rate = dragon_souls['result'].mean()
observed_mean, size, win_rate

In [ ]:
test_means = []

# The hypothesis test
for _ in range(100000):
    # Result is randomly chosen to be True or False according to the overall win rate 0.7936
    simulation = np.random.choice([True, False], size=size, p=[win_rate, 1 - win_rate])
    test_mean = simulation.mean()
    test_means.append(test_mean)

test_means[:10]

In [ ]:
pval = (observed_mean >= np.array(test_means)).mean()
pval

So we got our P-value, `0.04478`, which is **smaller** than our **significance level 0.05**. Therefore we **failed to reject** our null hypothesis. In other words, we are not confident that Cloud Dragon soul has a lower win rate simply because of random chance. Probably it is true that it is less helpful for winning the game, and cause the Riot Game released the rebalance patch?